Ta sẽ cho dữ liệu của rotten tomato vào data warehouse, với schema được thể hiện như sau:

Bảng Movie Info: chứa các thông tin chung của movie

- movie_id: mã phim, được mã hoá dưới dạng số
- movie_title: tiêu đề
- movie_info: mô tả
- content_rating: giới hạn độ tuổi xem
- genres: thể loại
- directors: đạo diễn bộ phim
- actors: diễn viên tham gia bộ phim
- release_date: ngày ra mắt (chỉ lấy năm, để ở dạng float)
- runtime: thời lượng bộ phim

Bảng Tomato Rating Info: chứa các thông tin về rating của movie trên trang Rotten Tomato
- movie_id: khoá ngoại
- tomatometer_rating: rating trung bình của các nhà phê bình phim
- tomatometer_count: số lượng rating của các nhà phê bình phim
- audience_rating: rating trung bình của khán giả
- audience_count: số lượng rating của khán giả
- link: link của trang web dẫn đến phim

Bảng Tomato Review: chứa các thông tin bình luận và đánh giá của các nhà phê bình phim
- movie_id: khoá ngoại
- critic_name: tên người đánh giá
- publisher_name: tên nhà sản xuất đánh giá
- review_type: loại review (Fresh/Rotten)
- review_score: điểm đánh giá
- review_date: thời gian đánh giá
- review_content: nội dung đánh giá

Với các nguồn dữ liệu khác khi cho vào data warehouse cũng sẽ tách ra làm 2 bảng riêng: bảng về thông tin chung về movie và bảng về thông tin rating của movie đối với nguồn đó

Ngoài ra, còn một số bảng tham khảo các giá trị của các trường trong data warehouse thể hiện ở dưới (Phục vụ cho xử lý các nguồn dữ liệu khác):

- Bảng Cast and Director: chứa danh sách các đạo diễn và diễn viên tham gia phim
- Bảng Genres: chứa danh sách các thể loại
- Bảng Content Rating: chứa danh sách các content rating

In [2]:
import pandas as pd
import re

In [44]:
# Movie Info
df_tomato_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/rotten_tomatoes/rotten_tomatoes_movies.csv")
df_tomato_movie.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [45]:
# Review Info
df_tomato_review = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/rotten_tomatoes/rotten_tomatoes_critic_reviews.csv")
df_tomato_review.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


# Xử lý dữ liệu và tạo bảng trong data warehouse

## Xử lý dữ liệu

### Tạo khoá cho dữ liệu

Do nguồn rotten tomato không có id phim riêng, cho nên ta xem xét việc lấy trường rotten_tomatoes_link làm liên kết tạm thời giữa hai bảng movie và review

In [46]:
print("Số bản ghi trong df_tomato_movie có trường rotten_tomatoes_link null: "+ str(sum(df_tomato_movie["rotten_tomatoes_link"].isna())))
print("Số bản ghi trong df_tomato_review có trường rotten_tomatoes_link null: "+ str(sum(df_tomato_review["rotten_tomatoes_link"].isna())))

Số bản ghi trong df_tomato_movie có trường rotten_tomatoes_link null: 0
Số bản ghi trong df_tomato_review có trường rotten_tomatoes_link null: 0


In [47]:
# Tạo một trường movie_id
df_tomato_movie["movie_id"]=["movie"+str(idx) for idx in range(len(df_tomato_movie))]
# Chỉnh sửa trường rotten_tomatoes_link
df_tomato_review["rotten_tomatoes_link"]=["https://www.rottentomatoes.com/"+i for i in df_tomato_review["rotten_tomatoes_link"]]
df_tomato_movie["rotten_tomatoes_link"]=["https://www.rottentomatoes.com/"+i for i in df_tomato_movie["rotten_tomatoes_link"]]

Ta sẽ tạo thêm cột movie_id cho bảng review

In [48]:
# Tạo dictionary liên kết movie_id và rotten_tomatoes_link
link_id_dict = {}
for i in range(len(df_tomato_movie)):
    link_id_dict[df_tomato_movie.iloc[i]["rotten_tomatoes_link"]]=df_tomato_movie.iloc[i]["movie_id"]

In [49]:
def create_movie_id(link):
    # Có khả năng link đó không tồn tại trong bảng movie
    if link in link_id_dict.keys():
        return link_id_dict[link]
    else:
        return None
df_tomato_review["movie_id"]=[create_movie_id(i) for i in df_tomato_review["rotten_tomatoes_link"]]

In [50]:
print("Số đường link không rõ trong trường rotten_tomatoes_link của bảng review: "+str(sum(df_tomato_review["movie_id"].isna())))

Số đường link không rõ trong trường rotten_tomatoes_link của bảng review: 130


In [51]:
# Tiến hành loại bỏ các bản ghi đó
df_tomato_review = df_tomato_review.dropna(subset=["movie_id"])

### Chuyển đổi release_date

In [52]:
df_warehouse_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/movie.csv")

In [53]:
df_tomato_movie["original_release_date"].head()

0    2010-02-12
1    2010-04-30
2    1979-10-05
3    1957-04-13
4    1954-01-01
Name: original_release_date, dtype: object

Ta thấy giá trị trường release_date có dạng YYYY-mm-DD, ta sẽ chỉ lấy year

In [54]:
def get_year(date):
    if type(date)!=float:
        return float(date[:4])
df_tomato_movie["original_release_date"]=[get_year(i) for i in df_tomato_movie["original_release_date"]]

### Chuyển đổi review_score

In [56]:
df_tomato_review["review_score"].unique()

array([nan, '3.5/5', '1/4', 'B', '3/5', '4/5', '2/4', '2/5', 'C', '2.5/4',
       '3/4', 'C-', '2.75/5', 'B-', '2.5/5', '5.5/10', '1/5', 'C+', 'D+',
       '1.5/4', '6.5/10', '3/10', '1.5/5', '4/10', '7/10', '6/10', 'B+',
       '5/5', 'A-', '4.5/5', '5.78/10', '3.5/4', 'A', '9/10', '8.5/10',
       '4/4', '83/100', '8/10', '92/100', '82/100', '84/100', '10/10',
       'D-', 'F', '0.5/4', 'D', '5/10', '2/10', '2/6', '0.5/5', '0/4',
       '0/5', '76/100', '87/100', '74/100', '86/100', '81/100', '56/100',
       '78/100', '2.25/5', '8.8/10', '0/10', '44/100', '2.75/4', '2.4/5',
       '8.6/10', '79/100', '68/100', '80/100', '90/100', '66/100',
       '91/100', '0.3/5', '9.2/10', '2.7/5', '57/100', '4/6', '3/6',
       '73/100', '42/100', '65/100', '7.5/10', '4.5/10', '5/6', '1/6',
       '70/100', '3.5/10', '71/100', '0.1/5', '3.25/10', '89/100',
       '40/100', '64/100', '62/100', '1/10', '3.75/5', '37/100', '49/100',
       '60/100', '3.3/5', '77/100', '5.25/10', '1.4/4', '75/100', '

Ta thấy review score rất đa dạng, như score/max_score, score, alphabet score (A,B,C, ...)

In [57]:
count=0
for i in df_tomato_review["review_score"]:
    if type(i)==str:
        if not bool(re.search(r'\d', i)):
            count+=1
print("% Số bản ghi review_score có dạng alphabet: "+str(count/len(df_tomato_review)))
print("% Số bản ghi review_score bị null: "+str(count/sum(df_tomato_review["review_score"].isna())))

% Số bản ghi review_score có dạng alphabet: 0.11409017007895479
% Số bản ghi review_score bị null: 0.42140620198625706


In [58]:
# Score dạng alphabet
for i in df_tomato_review["review_score"].unique():
    if type(i)==str:
        if not bool(re.search(r'\d', i)):
            print(i)

B
C
C-
B-
C+
D+
B+
A-
A
D-
F
D
A  -
C  -


Ta sẽ chuyển về từ dạng alphabet score sang dạng số (100) (Lấy trung bình) (https://www.cs.uni.edu/~mccormic/lettergrade.html):

|Percent|Letter Grade|
|-|-|
|97|A|
|91.5|A-|
|88|B+|
|84.5|B|
|81|B-|
|78|C+|
|74.5|C|
|71|C-|
|68|D+|
|64.5|D|
|61|D-|
|30|F|

In [59]:
alphabet_alphabet_dict = {"A  -": "A-", "C  -": "C-"}
def alphabet_to_numeric(score):
    # F
    if score=="F":
        return 30
    # D
    elif score=="D-":
        return 61
    elif score=="D":
        return 64.5
    elif score=="D+":
        return 68
    # C
    elif score=="C-":
        return 71
    elif score=="C":
        return 74.5
    elif score=="C+":
        return 78
    # B
    elif score=="B-":
        return 81
    elif score=="B":
        return 84.5
    elif score=="B+":
        return 88
    # A
    elif score=="A-":
        return 91.5
    elif score=="A":
        return 97

def convert_score(score):
    if type(score)==float:
        return None
    if type(score)==str:
        # Nếu điểm là dạng alphabet => chuyển về dạng số
        if not bool(re.search(r'\d', score)):
            if score in alphabet_alphabet_dict.keys():
                return alphabet_to_numeric(alphabet_alphabet_dict[score])
            else:
                return alphabet_to_numeric(score)
        else:
            if "/" in score:
                try:
                    return eval(score)*100
                except ZeroDivisionError:
                    return None
            else:
                return eval(score)

df_tomato_review["review_score"]=[convert_score(i) for i in df_tomato_review["review_score"]]

In [60]:
# Loại bỏ các trường không xuất hiện trong warehouse schema
df_tomato_movie.drop(['critics_consensus', 'authors', 'streaming_release_date', 
                      'tomatometer_status', 'audience_status', 'tomatometer_top_critics_count', 
                      'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count', 'production_company'], axis=1, inplace=True)
df_tomato_review.drop(["rotten_tomatoes_link"], axis=1, inplace=True)

In [61]:
# Thay đổi tên một số trường
df_tomato_movie.rename(columns = {'rotten_tomatoes_link':'link', 'original_release_date':'release_date'}, inplace = True)
df_tomato_movie.columns

Index(['link', 'movie_title', 'movie_info', 'content_rating', 'genres',
       'directors', 'actors', 'release_date', 'runtime', 'tomatometer_rating',
       'tomatometer_count', 'audience_rating', 'audience_count', 'movie_id'],
      dtype='object')

In [38]:
df_tomato_review.columns

Index(['critic_name', 'top_critic', 'publisher_name', 'review_type',
       'review_score', 'review_date', 'review_content', 'movie_id'],
      dtype='object')

## Tạo bảng trong datawarehouse

In [62]:
df_warehouse_movie = df_tomato_movie[["movie_id", "movie_title", "movie_info", "genres", 
                                      "directors", "actors", "release_date", "runtime", "content_rating"]]
df_warehouse_tomato_rating = df_tomato_movie[["movie_id", "tomatometer_rating", "tomatometer_count", 
                                              "audience_rating", "audience_count", "link"]]

# Tạo các bảng tham khảo

## Trường content_rating

In [8]:
df_warehouse_movie['content_rating'].unique()

array(['PG', 'R', 'NR', 'G', 'PG-13', 'NC17'], dtype=object)

content_rating description:

- G: GENERAL AUDIENCES: ALL AGES ADMITTED
- PG: PARENTAL GUIDANCE SUGGESTED: SOME MATERIAL MAY NOT BE SUITABLE FOR CHILDREN
- PG-13: PARENTS STRONGLY CAUTIONED: SOME MATERIAL MAY BE INAPPROPRIATE FOR CHILDREN UNDER 13
- R: RESTRICTED: UNDER 17 REQUIRES ACCOMPANYING PARENT OR ADULT GUARDIAN
- NC-17: NO ONE 17 AND UNDER ADMITTED
- NR: NOT RATED: THE CONTENT OF THIS FILM HAS NOT BEEN EVALUATED (TRAILER)

In [9]:
pd.DataFrame(list(df_warehouse_movie['content_rating'].unique()), columns=["content_rating"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/content_rating.csv",
    index=False 
)

## Trường genres

Cấu trúc giá trị trong trường genres là <genre 1>,  <genres 2>, ...

In [11]:
genres_list = []
for i in df_warehouse_movie["genres"]:
    if type(i)==str:
        genres_items = i.split(", ")
        genres_list += genres_items
genres_list = list(set(genres_list))

In [14]:
genres_list

['Art House & International',
 'Sports & Fitness',
 'Television',
 'Science Fiction & Fantasy',
 'Comedy',
 'Musical & Performing Arts',
 'Anime & Manga',
 'Mystery & Suspense',
 'Drama',
 'Cult Movies',
 'Western',
 'Faith & Spirituality',
 'Horror',
 'Kids & Family',
 'Documentary',
 'Animation',
 'Classics',
 'Special Interest',
 'Action & Adventure',
 'Gay & Lesbian',
 'Romance']

In [34]:
pd.DataFrame(genres_list, columns=["genres"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv",
    index=False 
)

## Trường directors và actors

Cấu trúc giá trị của trường director <director 1>, <director 2>, ...

Cấu trúc giá trị của trường actor <actor 1>, <actor 2>, ...


In [63]:
name_list = []
for i in df_warehouse_movie["directors"]:
    if type(i)==str:
        name_items = i.split(", ")
        name_list += name_items
for i in df_warehouse_movie["actors"]:
    if type(i)==str:
        name_items = i.split(", ")
        name_list += name_items
name_list = list(set(name_list))
name_list.sort()

In [39]:
len(name_list)

210926

In [40]:
pd.DataFrame(name_list, columns=["name"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/directors_actors.csv",
    index=False 
)

# Lưu dữ liệu data warehouse

In [65]:
df_warehouse_movie.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/movie.csv", index=False)
df_warehouse_tomato_rating.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/tomato_rating.csv", index=False)
df_tomato_review.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/tomato_review.csv", index=False)